<a href="https://colab.research.google.com/github/pratikunterwegs/elemove/blob/master/temp_kruger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as geemap
except:
    import geemap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

In [ ]:
# prelim code for earth engine landcover around duke

# get transects from user assets
transects = ee.FeatureCollection("users/pratik_unterwegs/squirrel_transect_bbox")

# get a 1km buffer
transect_geom = transects.geometry()

# define start and end date
start_date = '2020-04-01'
end_date = '2020-10-01'

# filter sentinel by bounding box
# get sentinel data
sentinel = ee.ImageCollection("COPERNICUS/S2")
sentinel_bounds = sentinel.filterBounds(transect_geom)
sentinel_2020 = sentinel_bounds.filterDate(start_date, end_date)

# adding a NDVI band
def add_ndvi(image):
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('ndvi')
    return image.addBands([ndvi])


# add an NDBI band
def add_ndbi(image):
    ndbi = image.normalizedDifference(['B11', 'B8']).rename('ndbi')
    return image.addBands([ndbi])


# map metrics over sentinel
sentinel_ndvi = sentinel_2020.map(add_ndvi)
sentinel_ndbi = sentinel_2020.map(add_ndbi)

# get median values
ndvi_median = sentinel_ndvi.select('ndvi').median()
ndbi_median = sentinel_ndbi.select('ndbi').median()

# clip to geometry
ndvi_clip = ndvi_median.clip(transect_geom)
ndbi_clip = ndbi_median.clip(transect_geom)

# get built up prob as ndbi - ndvi
built = ndbi_median.subtract(ndvi_median)
built_clip = built.clip(transect_geom)


In [ ]:
# Define an sld style color ramp to apply to the image.
sld_ramp = \
  '<RasterSymbolizer>' + \
    '<ColorMap type="ramp" extended="false" >' + \
      '<ColorMapEntry color="#0D0887" quantity="-1" label="0"/>' + \
      '<ColorMapEntry color="#CC4678" quantity="0" label="300" />' + \
      '<ColorMapEntry color="#F0F921" quantity="0.5" label="500" />' + \
    '</ColorMap>' + \
  '</RasterSymbolizer>';

# print layers
vis = {'bands': ['ndvi']}
Map = geemap.Map(center=[36.0005,-78.9], zoom=12)
Map.addLayer(built_clip.sldStyle(sld_ramp), {}, 'ndvi')
Map.addLayer(transects)
Map.addLayerControl()

Map

In [ ]:
# export image to drive
downConfig = {'scale': 10, "maxPixels": 1.0E13, 'driveFolder': 'duke-squirrels'}  # scale means resolution.
name = "duke_built_up"
# print(name)
task = ee.batch.Export.image(built_clip, name, downConfig)
task.start()